In [1]:
#establishing environment
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr, ttest_rel, ttest_ind
from scipy import signal as sig
import matplotlib.patches as mpatches
import mat73
from scipy.io import loadmat, savemat
import mne
#conda install git+https://github.com/ieeg-portal/ieegpy.git # Install ieegpy toolbox directly from github
from ieeg.auth import Session
import random
from sklearn.cluster import KMeans
import os
os.chdir('/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/functions')
from ied_functions import *

Identify the regions that we want to look at in order to establish some differences.

- SOZ vs. nonSOZ
- cortex vs non-cortex
- cortex vs. subcortical
- subcortical vs. subcortical
- depth vs. grid/strip

ALSO: atlas agnostic method in which we form clusters of about 5mm, group electrodes, and create heatmap of morphological feature.

FILES:
- all_ptids.csv (masterfile, contains r_id, hup_id, whichPts, ptname[same as hup_id])
- pt_ids.csv (raw r_id, hup_id)
- ptname_all.csv (whichPts, ptname [examples: ''HUP100''])
- SOZ_channels.csv (all SOZ electrodes for each pt - however only up to the whichPts = 1:119)
- RID.txt (all rids listed)

In [2]:
data_directory = '/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database'
ptnames = ['HUP105','HUP106','HUP107','HUP111']

In [3]:
spike, brain_df, ids = load_ptall(ptnames[0],data_directory)
print(ids)

['HUP105', 'sub-RID0101']


#### TO DO
- ~~what brain region do we want~~
- ~~add "-CAR" to the chlabels~~
- ~~find chlabel that match~~
- match it with spike.chlabel
- run amplitude/linelength to the values of these spikes
- First decision: only choose those that start with this channel, or run it for all values.
- then make a bar plot to graph
- run it for all patients.

In [5]:
pd.set_option('display.max_rows', 200)
brain_df


,Unnamed: 0,name,x,y,z,index,label
0,0,LA1-CAR,77.7201,131.6120,51.0434,1006.0,left entorhinal
1,1,LA2-CAR,72.6069,131.7390,50.8534,0.0,EmptyLabel
2,2,LA3-CAR,67.4461,131.5880,51.6281,0.0,EmptyLabel
3,3,LA4-CAR,62.3330,131.7140,51.4380,0.0,EmptyLabel
4,4,LAT1-CAR,71.6354,146.6290,42.1220,1006.0,left entorhinal
5,5,LAT2-CAR,65.3583,138.5490,37.7429,1007.0,left fusiform
6,6,LAT3-CAR,57.3199,132.7050,35.9230,1009.0,left inferior temporal
7,7,LAT4-CAR,48.5460,127.4160,38.3569,0.0,EmptyLabel
8,8,LF1-CAR,41.5512,93.2913,116.5260,0.0,EmptyLabel
9,9,LF2-CAR,35.1300,96.7684,108.8310,1031.0,left supramarginal


In [17]:
roi = ' left entorhinal '
roi_ch = brain_df.loc[brain_df['label']== roi]

display(roi_ch)
roi_chlist = np.array(roi_ch['name'])

,Unnamed: 0,name,x,y,z,index,label
0,0,LA1-CAR,77.7201,131.612,51.0434,1006.0,left entorhinal
4,4,LAT1-CAR,71.6354,146.629,42.1220,1006.0,left entorhinal
14,14,LH1-CAR,73.7539,126.223,47.4417,1006.0,left entorhinal


In [18]:
print(roi_chlist)

['LA1-CAR' 'LAT1-CAR' 'LH1-CAR']


In [56]:
idx_roich = []
for ch in roi_chlist:
    x = np.where(spike.chlabels[0][0] == ch)[0]
    idx_roich.append(x)


In [57]:
idx_roich = [x[0] for x in idx_roich]
idx_roich = [x+1 for x in idx_roich]
print(idx_roich)

#add 1, to get the true chlabel, which should be similar to the chlabel idx

[2, 6, 16]


In [61]:
counts,chs = hifreq_ch_spike(spike.select)
print(counts)

#use old function to generate counts, chs -> this will provide information of where the spike detector detected these channels of interest

[array([ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 15., 16.,
       17., 18., 19., 20., 21., 22., 23., 28., 29., 30., 31., 32., 33.,
       34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46.,
       47., 48., 50., 51., 52., 53., 54., 55., 56., 57., 58., 60.]), [37, 48, 49, 13, 4, 10, 2, 9, 3, 2, 2, 1, 32, 11, 16, 20, 6, 9, 15, 1, 2, 6, 8, 18, 149, 165, 144, 56, 19, 12, 3, 1, 6, 8, 21, 14, 1, 2, 4, 6, 4, 11, 23, 11, 7, 2, 3, 1, 1, 1, 1]]


In [62]:
select_oi = []
for chroi in idx_roich:
    idx = np.where(chs == chroi)[0]
    select_oi.append(idx)
print(select_oi)

#select_oi finds where it is on our 1000 random spikes that we detected the channel !first!

[array([ 10,  46,  69,  85,  90, 100, 119, 150, 212, 222, 225, 231, 251,
       258, 260, 266, 279, 327, 330, 403, 472, 491, 563, 650, 690, 703,
       704, 705, 714, 739, 744, 756, 761, 811, 886, 902, 959]), array([ 29, 228, 672, 971]), array([  2,  43,  50,  54, 108, 136, 235, 248, 254, 334, 357, 383, 417,
       445, 497, 516, 540, 551, 558, 585, 670, 731, 735, 757, 778, 779,
       834, 844, 850, 864, 904, 910])]


In [67]:
#sanity check for the select_oi
#if we were right the spike.select should have the value 2, for the 10th idx
print(spike.select[10][1])
print(spike.select[29][1])
print(spike.select[2][1])

2.0
6.0
16.0


In [85]:
#np.size(select_oi[0]) + np.size(select_oi[1]) + np.size(select_oi[2])


73

In [93]:
#select_oi = np.concatenate(select_oi)
select_oi = [int(x) for x in select_oi]
values_oi = []
for soi in select_oi:
    values_oi.append(spike.values[soi])

print(np.shape(values_oi))
#values_oi will contain thevalues that we require for our calculations. Essentially splitting it up.

(73, 2001, 61)


In [104]:
def value_basis(spike, brain_df, roi):
    """
    Function that takes in all values, the DKT atlas dataframe, and a region of interest (ROI)
    returns a tailored, truncated list of the all the values given a specific ROI
    """
    roi_ch = brain_df.loc[brain_df['label']== roi] #creates truncated dataframe of ROI labels
    roi_chlist = np.array(roi_ch['name']) #converts DF to array

    #finds the index of where to find the channel
    idx_roich = []
    for ch in roi_chlist:
        x = np.where(spike.chlabels[0][0] == ch)[0]
        idx_roich.append(x)

    idx_roich = [x[0] for x in idx_roich]
    idx_roich = [x+1 for x in idx_roich]
    
    counts,chs = hifreq_ch_spike(spike.select)

    select_oi = []
    for chroi in idx_roich:
        idx = np.where(chs == chroi)[0]
        select_oi.append(idx)

    select_oi = np.concatenate(select_oi)
    select_oi = [int(x) for x in select_oi]
    values_oi = []
    for soi in select_oi:
        values_oi.append(spike.values[soi])

    based_values = values_oi
    
    return based_values

In [106]:
roi = ' left entorhinal '
based_values = value_basis(spike, brain_df, roi)

In [107]:
np.shape(based_values)

(73, 2001, 61)